## Data set Details

The data set is from the [Kasandr UCI data set](https://archive.ics.uci.edu/ml/datasets/KASANDR#).  This is a public data set.  Per the attribution policies, we cite the following paper for this data set:

    Sumit Sidana, Charlotte Laclau, Massih-Reza Amini, Gilles Vandelle, and Andre Bois-Crettez. 'KASANDR: A Large-Scale Dataset with Implicit Feedback for Recommendation', SIGIR 2017. 

The data set is already divided into train and test sets.  The train set has 15,844,718 samples.  The test set has 1,919,562 samples.  That's a total of 17,764,280 samples, with a 90/10 train/test split.

The schema is:

* Features
    * userid: Categorical
    * offerid: Categorical
    * countrycode: Categorical
    * category: Integer (but seems to be encoded, so we'll treat it as categorical)
    * merchant: Integer (but seems to be encoded, so we'll treat it as categorical)
    * utcdate: Timestamp, format '2016-06-01 02:00:17.0'
*  Label: promotion generated response is given in Rating: Binary, 0 or 1 


# 1) Data Analysis using Pyspark

## Data Loading

In [6]:
#import and create sparksession object
import time
from pyspark.sql import SparkSession 
# sc = SparkContext(master="local[2]")
# spark = SparkSession.builder.appName('rc').getOrCreate()
# spark = SparkSession.builder.master("local[2]").appName("rc").config("spark.some.config.option", "some-value").getOrCreate()
spark = SparkSession.builder.master("local[2]").appName("rc").getOrCreate()

In [5]:
spark.stop()

In [7]:
#Read data
dpath="C:\\Datasets\\MLData\\Kasandr\\dataset\\train_de.csv"
start = time.time()
#import the required functions and libraries
from pyspark.sql.functions import *
#load the dataset and create sprk dataframe
df=spark.read.csv(dpath,inferSchema=True,header=True,sep='\t')
end = time.time()
print(str(end - start)+' sec')

98.44799494743347 sec


In [3]:
df.show(5)

+--------------------+--------------------+-----------+---------+--------------------+-------------------+------+
|              userid|             offerid|countrycode| category|            merchant|            utcdate|rating|
+--------------------+--------------------+-----------+---------+--------------------+-------------------+------+
|fa937b779184527f1...|c5f63750c2b5b0166...|         de|100020213|f3c93baa0cf443084...|2016-06-14 17:28:47|     0|
|f6c8958b9bc2d6033...|19754ec121b3a99ff...|         de|100020213|21a509189fb0875c3...|2016-06-14 17:28:48|     0|
|02fe7ccf1de19a387...|5ac4398e4d8ad4167...|         de|   125801|b042951fdb45ddef8...|2016-06-14 17:28:50|     0|
|9de5c06d0a16256b1...|be83df9772ec47fd2...|         de|   125801|4740b6c83b6e12e42...|2016-06-14 17:29:19|     0|
|8d26ade603ea5473c...|3735290a415dc236b...|         de|   125801|8bf8f87492a799528...|2016-06-14 17:29:31|     0|
+--------------------+--------------------+-----------+---------+--------------------+--

In [6]:
#validate the shape of the data 
print((df.count(),len(df.columns)))

(15844717, 7)


In [14]:
#Target variable
target = df.select('rating').groupBy('rating').count()
target.show()

total_yes = target.head(2)[0][1] 
total_no = target.head(2)[1][1] 

+------+--------+
|rating|   count|
+------+--------+
|     1|  705447|
|     0|15139270|
+------+--------+



In [19]:
print("Target : {0:.2}%".format((total_yes/total_no)*100))

Target : 4.7%


In [20]:
df.select('countrycode').distinct().show()

+-----------+
|countrycode|
+-----------+
|         de|
+-----------+



* The country code is not useful, as the entire data set is for one country.

## Data Cleaning

In [21]:
df.columns

['userid',
 'offerid',
 'countrycode',
 'category',
 'merchant',
 'utcdate',
 'rating']

In [22]:
#Check for missing values
for col in df.columns:    
    print(col, ': ', df.filter((df[col] == "") | df[col].isNull()).count())

userid :  0
offerid :  0
countrycode :  0
category :  0
merchant :  0
utcdate :  0
rating :  0


## EDA

In [23]:
#Dates 
min_date, max_date = df.select(min("utcdate"), max("utcdate")).first()
min_date, max_date

(datetime.datetime(2016, 6, 1, 2, 0, 17),
 datetime.datetime(2016, 6, 14, 23, 52, 51))

In [8]:
df=df.drop('countrycode')
df.show(5)

+--------------------+--------------------+---------+--------------------+-------------------+------+
|              userid|             offerid| category|            merchant|            utcdate|rating|
+--------------------+--------------------+---------+--------------------+-------------------+------+
|fa937b779184527f1...|c5f63750c2b5b0166...|100020213|f3c93baa0cf443084...|2016-06-14 17:28:47|     0|
|f6c8958b9bc2d6033...|19754ec121b3a99ff...|100020213|21a509189fb0875c3...|2016-06-14 17:28:48|     0|
|02fe7ccf1de19a387...|5ac4398e4d8ad4167...|   125801|b042951fdb45ddef8...|2016-06-14 17:28:50|     0|
|9de5c06d0a16256b1...|be83df9772ec47fd2...|   125801|4740b6c83b6e12e42...|2016-06-14 17:29:19|     0|
|8d26ade603ea5473c...|3735290a415dc236b...|   125801|8bf8f87492a799528...|2016-06-14 17:29:31|     0|
+--------------------+--------------------+---------+--------------------+-------------------+------+
only showing top 5 rows



In [9]:
df = df.withColumn('day_of_week',dayofweek(df.utcdate))
df.show(5)

+--------------------+--------------------+---------+--------------------+-------------------+------+-----------+
|              userid|             offerid| category|            merchant|            utcdate|rating|day_of_week|
+--------------------+--------------------+---------+--------------------+-------------------+------+-----------+
|fa937b779184527f1...|c5f63750c2b5b0166...|100020213|f3c93baa0cf443084...|2016-06-14 17:28:47|     0|          3|
|f6c8958b9bc2d6033...|19754ec121b3a99ff...|100020213|21a509189fb0875c3...|2016-06-14 17:28:48|     0|          3|
|02fe7ccf1de19a387...|5ac4398e4d8ad4167...|   125801|b042951fdb45ddef8...|2016-06-14 17:28:50|     0|          3|
|9de5c06d0a16256b1...|be83df9772ec47fd2...|   125801|4740b6c83b6e12e42...|2016-06-14 17:29:19|     0|          3|
|8d26ade603ea5473c...|3735290a415dc236b...|   125801|8bf8f87492a799528...|2016-06-14 17:29:31|     0|          3|
+--------------------+--------------------+---------+--------------------+--------------

In [10]:
df.select('day_of_week').groupBy('day_of_week').count().show()

+-----------+-------+
|day_of_week|  count|
+-----------+-------+
|          1|2991702|
|          6|1923578|
|          3|2103404|
|          5|2403321|
|          4|1955536|
|          7|2412142|
|          2|2055034|
+-----------+-------+



In [10]:
df = df.withColumn('hour',hour(df.utcdate))
df.show(5)

+--------------------+--------------------+---------+--------------------+-------------------+------+-----------+----+
|              userid|             offerid| category|            merchant|            utcdate|rating|day_of_week|hour|
+--------------------+--------------------+---------+--------------------+-------------------+------+-----------+----+
|fa937b779184527f1...|c5f63750c2b5b0166...|100020213|f3c93baa0cf443084...|2016-06-14 17:28:47|     0|          3|  17|
|f6c8958b9bc2d6033...|19754ec121b3a99ff...|100020213|21a509189fb0875c3...|2016-06-14 17:28:48|     0|          3|  17|
|02fe7ccf1de19a387...|5ac4398e4d8ad4167...|   125801|b042951fdb45ddef8...|2016-06-14 17:28:50|     0|          3|  17|
|9de5c06d0a16256b1...|be83df9772ec47fd2...|   125801|4740b6c83b6e12e42...|2016-06-14 17:29:19|     0|          3|  17|
|8d26ade603ea5473c...|3735290a415dc236b...|   125801|8bf8f87492a799528...|2016-06-14 17:29:31|     0|          3|  17|
+--------------------+--------------------+-----

In [11]:
df=df.drop('utcdate')
df.show(5)

+--------------------+--------------------+---------+--------------------+------+-----------+----+
|              userid|             offerid| category|            merchant|rating|day_of_week|hour|
+--------------------+--------------------+---------+--------------------+------+-----------+----+
|fa937b779184527f1...|c5f63750c2b5b0166...|100020213|f3c93baa0cf443084...|     0|          3|  17|
|f6c8958b9bc2d6033...|19754ec121b3a99ff...|100020213|21a509189fb0875c3...|     0|          3|  17|
|02fe7ccf1de19a387...|5ac4398e4d8ad4167...|   125801|b042951fdb45ddef8...|     0|          3|  17|
|9de5c06d0a16256b1...|be83df9772ec47fd2...|   125801|4740b6c83b6e12e42...|     0|          3|  17|
|8d26ade603ea5473c...|3735290a415dc236b...|   125801|8bf8f87492a799528...|     0|          3|  17|
+--------------------+--------------------+---------+--------------------+------+-----------+----+
only showing top 5 rows



### Column- day_of_week

In [8]:
#Weekdays Grouping 
weekdays = df.groupBy('day_of_week').agg({'rating': 'mean', 'day_of_week': 'count'})
weekdays.show()

+-----------+--------------------+------------------+
|day_of_week|         avg(rating)|count(day_of_week)|
+-----------+--------------------+------------------+
|          1| 0.03237889335234592|           2991702|
|          6| 0.03882920266295414|           1923578|
|          3| 0.10241874599458782|           2103404|
|          5| 0.03401875987435719|           2403321|
|          4|0.041626950360412696|           1955536|
|          7|0.031137470347931424|           2412142|
|          2|0.039021738813080464|           2055034|
+-----------+--------------------+------------------+



In [11]:
weekdays = weekdays.withColumn('count(positive)', (col('avg(rating)') * col('count(day_of_week)')).cast('int'))
weekdays = weekdays.sort("avg(rating)", ascending=False)
weekdays.show()

+-----------+--------------------+------------------+---------------+
|day_of_week|         avg(rating)|count(day_of_week)|count(positive)|
+-----------+--------------------+------------------+---------------+
|          3| 0.10241874599458782|           2103404|         215428|
|          4|0.041626950360412696|           1955536|          81403|
|          2|0.039021738813080464|           2055034|          80191|
|          6| 0.03882920266295414|           1923578|          74691|
|          5| 0.03401875987435719|           2403321|          81758|
|          1| 0.03237889335234592|           2991702|          96868|
|          7|0.031137470347931424|           2412142|          75108|
+-----------+--------------------+------------------+---------------+



### Column - hour

In [12]:
#Hour Grouping 
hours = df.groupBy('hour').agg({'rating': 'mean', 'hour': 'count'})

hours = hours.withColumn('count(positive)', (col('avg(rating)') * col('count(hour)')).cast('int'))
hours = hours.sort("avg(rating)", ascending=False)
hours.show(24)

+----+-----------+--------------------+---------------+
|hour|count(hour)|         avg(rating)|count(positive)|
+----+-----------+--------------------+---------------+
|   0|      27285|  0.3720725673446949|          10152|
|  20|     124057|  0.3546514908469494|          43997|
|   1|      19329| 0.32019245693000153|           6189|
|  21|     140220| 0.31691627442590214|          44438|
|  22|     128301| 0.30285812269584805|          38857|
|  23|     139157|  0.2634003319991089|          36654|
|  19|     264503| 0.16856519585789198|          44586|
|  18|     581059| 0.08287970756842249|          48158|
|  17|    1275499| 0.04213723413346463|          53746|
|   7|     499146| 0.03726164288604937|          18599|
|  16|    1558179| 0.03268302293895631|          50926|
|  10|    1348360| 0.03207600344121748|          43250|
|   8|     771206|0.031104529788409323|          23988|
|   9|    1022504|0.029534358789794466|          30199|
|  11|    1330400| 0.02866882140709561|         

> This attribute shows that maximum positive counts is generally occuring at evening and night

### Column - Merchants

In [12]:
#Merchants Grouping 
mercs = df.groupBy('merchant').agg({'rating': 'mean', 'merchant': 'count'})

mercs = mercs.withColumn('count(positive)', (col('avg(rating)') * col('count(merchant)')).cast('int'))
mercs = mercs.sort("avg(rating)", ascending=False)
mercs.show(5)

+--------------------+------------------+---------------+---------------+
|            merchant|       avg(rating)|count(merchant)|count(positive)|
+--------------------+------------------+---------------+---------------+
|245d2c7b8e6fc4de4...|               1.0|             54|             54|
|3e0c8ff0db6c0ba0a...|               1.0|             31|             31|
|36e2130a3c07037b1...|0.9523809523809523|             21|             20|
|ea0c486f7e2afef05...|0.9354838709677419|             31|             29|
|cde1bb72b28d4a887...|0.9230769230769231|             13|             12|
+--------------------+------------------+---------------+---------------+
only showing top 5 rows



In [15]:
mercs.filter(mercs['count(positive)'] > 10000).show()
print('Total Yes: ', total_yes)


+--------------------+-------------------+---------------+---------------+
|            merchant|        avg(rating)|count(merchant)|count(positive)|
+--------------------+-------------------+---------------+---------------+
|66863da8db7e6c51b...| 0.6265489999207334|          75694|          47425|
|a7b2f269064dbe77e...|0.30314010305378086|         187669|          56890|
|ac26975cf46eae989...| 0.1618780124357797|         314737|          50949|
|70ea724342fb2d118...| 0.1512924051551128|          82714|          12514|
|eb49b22a1bbd88fbd...|0.07760938578329883|         181125|          14057|
|ab8863ef55e574c00...|0.06480676778336472|         579924|          37583|
|5878d16d0c0691283...| 0.0536400383381256|         239970|          12872|
|fca91704667a53350...|0.04780318920597915|         286529|          13697|
|8497a9dd86ab3b7f1...|0.04425939471683428|         277523|          12283|
|154f65f908a740682...| 0.0244791419290489|         484249|          11854|
+--------------------+---

> top three merchant have more then 21% of total clicks

### Column - Categories

In [16]:
#Categories Grouping 
cats = df.groupBy("category").agg({'rating': 'mean', 'category': 'count'})

cats = cats.withColumn('count(positive)', (col('avg(rating)') * col('count(category)')).cast('int'))
cats = cats.sort("avg(rating)", ascending=False)
cats.show(5)

+---------+------------------+---------------+---------------+
| category|       avg(rating)|count(category)|count(positive)|
+---------+------------------+---------------+---------------+
|100020813|               1.0|             54|             54|
|   121201|0.9902912621359223|            103|            102|
|100345723|0.9705882352941176|            170|            165|
|   128101|               0.9|             20|             18|
|100333423|0.8773584905660378|            212|            186|
+---------+------------------+---------------+---------------+
only showing top 5 rows



In [17]:
cats.filter(cats['count(positive)'] > 10000).show()
print('Total Yes: ', total_yes)

+---------+--------------------+---------------+---------------+
| category|         avg(rating)|count(category)|count(positive)|
+---------+--------------------+---------------+---------------+
|100010713|  0.5184821498016645|         102856|          53329|
|   125801|  0.1227365557895503|         439706|          53968|
|100020213| 0.08024719256855496|         663774|          53266|
|100354123| 0.07432864699549051|         769044|          57162|
|   130401|0.050088623711211526|         200285|          10032|
|100434023|0.049709758354654014|         202073|          10045|
|   134101| 0.04908641060584009|         226743|          11130|
|100091613| 0.04691772227138838|         302828|          14208|
|   142101| 0.04081416620078929|         302297|          12337|
|   168001| 0.03791937619216083|         274187|          10397|
|100232023| 0.03331888597215748|         366849|          12222|
|   113501|0.028335950614609844|         461675|          13082|
|   108301| 0.02050122659

> Interestingly top 4 make up more then 30% of all clicks

> Ques) **Does merchant no.1 sell cat no. 1?**

In [18]:
merch_id = mercs.filter(mercs['count(positive)'] > 10000).head(1)[0][0]

df.filter(df['merchant'] == merch_id).groupby('category').count().show()

+---------+-----+
| category|count|
+---------+-----+
|100010713|75694|
+---------+-----+



> Ans) **yes**

> Ques) **How about merchant No. 2?**

In [19]:
merch_id = mercs.filter(mercs['count(positive)'] > 10000).head(2)[1][0]

df.filter(df['merchant'] == merch_id).groupby('category').count().sort('count', ascending=False).show(10)

+---------+-----+
| category|count|
+---------+-----+
|100020213|68673|
|   125801|64063|
|   113501| 4770|
|   142101| 4712|
|   143101| 4562|
|   133301| 3953|
|   120901| 2963|
|     6513| 2770|
|100046613| 2180|
|100367723| 2136|
+---------+-----+
only showing top 10 rows



> Ans) **Heavily invoilved in category 2 & 3.**

### Column - User

In [20]:
#Users Grouping
users = df.groupBy('userid').agg({'rating': 'mean', 'userid': 'count'})

users = users.withColumn('count(positive)', (col('avg(rating)') * col('count(userid)')).cast('int'))
users = users.sort("avg(rating)", ascending=False)
users.show(5)

+--------------------+-----------+-------------+---------------+
|              userid|avg(rating)|count(userid)|count(positive)|
+--------------------+-----------+-------------+---------------+
|b30b99b5627c4b0f6...|        1.0|            6|              6|
|9e3dfcddda9bfd607...|        1.0|            5|              5|
|bfc7f5a747eec2abd...|        1.0|            2|              2|
|b112ac315de5042e0...|        1.0|            7|              7|
|84c3e6783aee36656...|        1.0|            2|              2|
+--------------------+-----------+-------------+---------------+
only showing top 5 rows



In [21]:
users.filter(users['count(positive)'] > 1000).show(10)

+--------------------+-------------------+-------------+---------------+
|              userid|        avg(rating)|count(userid)|count(positive)|
+--------------------+-------------------+-------------+---------------+
|6e2ab4134ce6b24d1...|0.30091743119266057|         4360|           1312|
|7511572a7068fe6e7...|                0.3|         3550|           1065|
|cad7e4a68117616ba...| 0.2992623814541623|         3796|           1136|
|7625efac4a89c43c4...|0.29914529914529914|         6318|           1890|
|4ba6cb76318d7db81...|0.29873150105708246|         4730|           1413|
|cd5fc9305c30dfd50...|0.29870441458733205|         4168|           1245|
|f0fbac7eb4c2c0ded...|0.29798870853916726|         5668|           1689|
|314dc010def122b88...| 0.2978501045088086|         6698|           1995|
|c08bfca37471d9c79...| 0.2968835429196282|         3658|           1086|
|ce57395c3fe3f1037...|0.29661354581673305|         5020|           1489|
+--------------------+-------------------+---------

> Got some power users here.

### Columns - Offers

In [22]:
#Offers Grouping
offers = df.groupBy('offerid').agg({'rating': 'mean', 'offerid': 'count'})

offers = offers.withColumn('count(positive)', (col('avg(rating)') * col('count(offerid)')).cast('int'))
offers = offers.sort("avg(rating)", ascending=False)
offers.show(10)

+--------------------+-----------+--------------+---------------+
|             offerid|avg(rating)|count(offerid)|count(positive)|
+--------------------+-----------+--------------+---------------+
|abf0b3ca05f7189d9...|        1.0|             1|              1|
|9532a4d7fc55ffa3e...|        1.0|             1|              1|
|cee894d4a0dcc6e8a...|        1.0|            23|             23|
|f68a72e3ae83b6996...|        1.0|             1|              1|
|98e998f0252bf5c55...|        1.0|             1|              1|
|4a2f4920a2c1d331d...|        1.0|             1|              1|
|5820537a3c82067fd...|        1.0|             1|              1|
|a338457b0c62a1743...|        1.0|             1|              1|
|20e5a52c452082acb...|        1.0|             1|              1|
|cbfa331093e73e1c4...|        1.0|             9|              9|
+--------------------+-----------+--------------+---------------+
only showing top 10 rows



In [23]:
offers.filter(offers['count(positive)'] > 100).show(10)

+--------------------+------------------+--------------+---------------+
|             offerid|       avg(rating)|count(offerid)|count(positive)|
+--------------------+------------------+--------------+---------------+
|a066572754a00f7a0...|0.9955156950672646|           223|            222|
|ae19aab375ac925bd...|0.9913793103448276|           232|            230|
|61ba4686f0a6b704a...|0.9911764705882353|           340|            337|
|c07db2553dc287e0d...|0.9902912621359223|           103|            102|
|e27570f3348a9fec4...|0.9883551673944687|           687|            679|
|5c2f1d277d6922bb7...|0.9870689655172413|           232|            229|
|ea268a1ab5ba20bdd...| 0.981203007518797|           266|            261|
|a1cb8243ccdbfda93...|0.9809523809523809|           105|            103|
|01c0d039ff17aa7f4...|0.9752066115702479|           121|            118|
|30a81c7e560b76f51...|0.9635036496350365|           137|            132|
+--------------------+------------------+----------

In [26]:
df.printSchema()

root
 |-- userid: string (nullable = true)
 |-- offerid: string (nullable = true)
 |-- category: integer (nullable = true)
 |-- merchant: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- hour: integer (nullable = true)



# 2) Model Building using pandas

## Model Building and Feature Engineering

In [ ]:
%cd /content/drive//My\ Drive/Notebook/ML_Chal_Kasandra/
## Google colab path set

In [1]:
#import the required functions and libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import pickle
%matplotlib inline

### Data load

In [16]:
#Read data
dpath="C:\\Datasets\\MLData\\Kasandr\\dataset\\train_de.csv"
start = time.time()
#load the dataset and create sprk dataframe
data=pd.read_csv(dpath,sep='\t')
end = time.time()
print("Data Loading time :",str(end - start)+' sec')
data.head()

Data Loading time : 73.14873099327087 sec


,userid,offerid,countrycode,category,merchant,utcdate,rating
0,fa937b779184527f12e2d71c711e6411236d1ab59f8597...,c5f63750c2b5b0166e55511ee878b7a3,de,100020213,f3c93baa0cf4430849611cedb3a40ec4094d1d370be841...,2016-06-14 17:28:47.0,0
1,f6c8958b9bc2d6033ff4c1cc0a03e9ab96df4bcc528913...,19754ec121b3a99fff3967646942de67,de,100020213,21a509189fb0875c3732590121ff3fc86da770b0628c18...,2016-06-14 17:28:48.0,0
2,02fe7ccf1de19a387afc8a11d08852ffd2b4dabaed4e2d...,5ac4398e4d8ad4167a57b43e9c724b18,de,125801,b042951fdb45ddef8ba6075ced0e5885bc2fa4c4470bf7...,2016-06-14 17:28:50.0,0
3,9de5c06d0a16256b13b8e7cdc50bf203ecef533eb5cbe1...,be83df9772ec47fd210b28091138ff11,de,125801,4740b6c83b6e12e423297493f234323ffd1c991f3d4496...,2016-06-14 17:29:19.0,0
4,8d26ade603ea5473c3844aebfcd9e96e6adc8ff411576e...,3735290a415dc236bacd7ed3aa03b2d5,de,125801,8bf8f87492a799528235c04bb18ff2d12db5058ff6e9a0...,2016-06-14 17:29:31.0,0


### Feature Engineering

In [32]:
data['utcdate']

0           2016-06-14 17:28:47.0
1           2016-06-14 17:28:48.0
2           2016-06-14 17:28:50.0
3           2016-06-14 17:29:19.0
4           2016-06-14 17:29:31.0
                    ...          
15844712    2016-06-14 21:12:14.0
15844713    2016-06-14 21:12:14.0
15844714    2016-06-14 21:12:19.0
15844715    2016-06-14 21:12:51.0
15844716    2016-06-14 21:12:51.0
Name: utcdate, Length: 15844717, dtype: object

In [17]:
###The day of the week with Monday=0, Sunday=6.
data['weekday'] = pd.to_datetime(data['utcdate']).apply(lambda x: x.weekday())
data.head()

,userid,offerid,countrycode,category,merchant,utcdate,rating,weekday
0,fa937b779184527f12e2d71c711e6411236d1ab59f8597...,c5f63750c2b5b0166e55511ee878b7a3,de,100020213,f3c93baa0cf4430849611cedb3a40ec4094d1d370be841...,2016-06-14 17:28:47.0,0,1
1,f6c8958b9bc2d6033ff4c1cc0a03e9ab96df4bcc528913...,19754ec121b3a99fff3967646942de67,de,100020213,21a509189fb0875c3732590121ff3fc86da770b0628c18...,2016-06-14 17:28:48.0,0,1
2,02fe7ccf1de19a387afc8a11d08852ffd2b4dabaed4e2d...,5ac4398e4d8ad4167a57b43e9c724b18,de,125801,b042951fdb45ddef8ba6075ced0e5885bc2fa4c4470bf7...,2016-06-14 17:28:50.0,0,1
3,9de5c06d0a16256b13b8e7cdc50bf203ecef533eb5cbe1...,be83df9772ec47fd210b28091138ff11,de,125801,4740b6c83b6e12e423297493f234323ffd1c991f3d4496...,2016-06-14 17:29:19.0,0,1
4,8d26ade603ea5473c3844aebfcd9e96e6adc8ff411576e...,3735290a415dc236bacd7ed3aa03b2d5,de,125801,8bf8f87492a799528235c04bb18ff2d12db5058ff6e9a0...,2016-06-14 17:29:31.0,0,1


In [18]:
data['hour'] = pd.DatetimeIndex(data['utcdate']).hour
data.head()

,userid,offerid,countrycode,category,merchant,utcdate,rating,weekday,hour
0,fa937b779184527f12e2d71c711e6411236d1ab59f8597...,c5f63750c2b5b0166e55511ee878b7a3,de,100020213,f3c93baa0cf4430849611cedb3a40ec4094d1d370be841...,2016-06-14 17:28:47.0,0,1,17
1,f6c8958b9bc2d6033ff4c1cc0a03e9ab96df4bcc528913...,19754ec121b3a99fff3967646942de67,de,100020213,21a509189fb0875c3732590121ff3fc86da770b0628c18...,2016-06-14 17:28:48.0,0,1,17
2,02fe7ccf1de19a387afc8a11d08852ffd2b4dabaed4e2d...,5ac4398e4d8ad4167a57b43e9c724b18,de,125801,b042951fdb45ddef8ba6075ced0e5885bc2fa4c4470bf7...,2016-06-14 17:28:50.0,0,1,17
3,9de5c06d0a16256b13b8e7cdc50bf203ecef533eb5cbe1...,be83df9772ec47fd210b28091138ff11,de,125801,4740b6c83b6e12e423297493f234323ffd1c991f3d4496...,2016-06-14 17:29:19.0,0,1,17
4,8d26ade603ea5473c3844aebfcd9e96e6adc8ff411576e...,3735290a415dc236bacd7ed3aa03b2d5,de,125801,8bf8f87492a799528235c04bb18ff2d12db5058ff6e9a0...,2016-06-14 17:29:31.0,0,1,17


In [19]:
df=data.drop(labels='utcdate',axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15844717 entries, 0 to 15844716
Data columns (total 8 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   userid       object
 1   offerid      object
 2   countrycode  object
 3   category     int64 
 4   merchant     object
 5   rating       int64 
 6   weekday      int64 
 7   hour         int64 
dtypes: int64(4), object(4)
memory usage: 967.1+ MB


In [20]:
y = df['rating']
X = df.drop(labels=['userid','rating'],axis=1)

In [21]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15844717 entries, 0 to 15844716
Data columns (total 6 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   offerid      object
 1   countrycode  object
 2   category     int64 
 3   merchant     object
 4   weekday      int64 
 5   hour         int64 
dtypes: int64(3), object(3)
memory usage: 725.3+ MB


In [22]:
from sklearn.preprocessing import OneHotEncoder
import pickle

In [23]:
# X_t = OneHotEncoder().fit_transform(X)
# print("Size of transformed data: {0}".format(X_t.shape))
##Size of transformed data: (15844717, 3114014)
#### To save the fit model ################
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X)

OneHotEncoder(handle_unknown='ignore')

In [24]:
filehandler = open("encoder.pkl","wb")
pickle.dump(enc,filehandler)
filehandler.close()

file = open("encoder.pkl",'rb')
enc= pickle.load(file)
file.close()
X_t=enc.transform(X)
print("Size of transformed data: {0}".format(X_t.shape))

Size of transformed data: (15844717, 2159865)


### Model Building without upsampling

In [21]:
from sklearn.model_selection import train_test_split
import catboost as cb
import lightgbm as lgb 
import pickle
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [44]:
train_x, test_x, train_y, test_y = train_test_split(X_t, y, test_size=0.10,random_state=42)

In [ ]:
############### LGBOOST Algorithm###############
small_lgb_model = lgb.LGBMClassifier()
small_lgb_model.fit(train_x, train_y)

In [ ]:
############## Model Save & Load ###########
filehandler = open("lgb_model.pkl","wb")
pickle.dump(enc,filehandler)
filehandler.close()

file = open("lgb_model.pkl",'rb')
lgb_model= pickle.load(file)
file.close()

In [24]:
# cheking initial test accuracy
y_pred = small_lgb_model.predict(test_x)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(test_y,predictions)
print('accuracy:',accuracy)

accuracy: 0.9685560868226135


#### Classification metrics

In [25]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, average_precision_score
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
from sklearn.metrics import precision_score, precision_recall_fscore_support, recall_score
# predvec = np.where(predictions > 0.5, 1, 0)
print('Weighted F1: {}'.format(f1_score(test_y, predictions,average='weighted')))
print('Accuracy: {}'.format(accuracy_score(test_y, predictions)))
print('Weighted ROC: {}'.format(roc_auc_score(test_y, predictions, average='weighted')))
print('Classification report: \n',classification_report(test_y, predictions),sep='')
# print('Confusion matrix: \n',confusion_matrix(test_y, predictions))

Weighted F1: 0.9632183052166432
Accuracy: 0.9685560868226135
Weighted ROC: 0.691292868185599
Classification report: 
              precision    recall  f1-score   support

           0       0.97      1.00      0.98   1513904
           1       0.81      0.39      0.52     70568

    accuracy                           0.97   1584472
   macro avg       0.89      0.69      0.75   1584472
weighted avg       0.96      0.97      0.96   1584472



### Model Building with upsampling

In [56]:
from imblearn.over_sampling import RandomOverSampler

In [58]:
rdsmple = RandomOverSampler()
x_sampled, y_sampled = rdsmple.fit_sample(X_t, y)
print("Size of sampled data: {0}".format(x_sampled.shape))

Size of sampled data: (30278540, 3114014)


In [35]:
train_x, test_x, train_y, test_y = train_test_split(x_sampled, y_sampled, test_size=0.25,random_state=42)

In [60]:
############### LGBOOST Algorithm###############
lgb_model = lgb.LGBMClassifier()
lgb_model.fit(train_x, train_y)

LGBMClassifier()

#### Classification metrics

In [61]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, average_precision_score
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
from sklearn.metrics import precision_score, precision_recall_fscore_support, recall_score
# cheking initial test results
y_pred = lgb_model.predict(test_x)
predictions = [round(value) for value in y_pred]
print('Weighted F1: {}'.format(f1_score(test_y, predictions,average='weighted')))
print('Accuracy: {}'.format(accuracy_score(test_y, predictions)))
print('Weighted ROC: {}'.format(roc_auc_score(test_y, predictions, average='weighted')))
print('Classification report: \n',classification_report(test_y, predictions),sep='')

Weighted F1: 0.7240969664362467
Accuracy: 0.734895011450354
Weighted ROC: 0.7348615922859637
Classification report: 
              precision    recall  f1-score   support

           0       0.67      0.93      0.78   3785457
           1       0.89      0.54      0.67   3784178

    accuracy                           0.73   7569635
   macro avg       0.78      0.73      0.72   7569635
weighted avg       0.78      0.73      0.72   7569635



In [1]:
# print('Confusion matrix: \n',confusion_matrix(test_y, predictions))

### Hyperparameter Tuning

In [1]:
from sklearn.model_selection import RandomizedSearchCV

In [2]:
param_test ={'num_leaves': [5, 10, 50], 
             'min_child_samples': [50, 100,500], 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e2],
             'subsample': [0.2,0.4,0.8], 
             'colsample_bytree': [0.2,0.4,0.6],
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [ ]:
fit_params={"early_stopping_rounds":10, 
            "eval_metric" : 'auc', 
            "eval_set" : [(test_x,test_y)],
            'eval_names': ['valid'],
            #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
            'verbose': 2,
            'categorical_feature': 'auto'}

In [ ]:
# #n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 1000 define only the absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=10)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    scoring='roc_auc',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

In [ ]:
gs.fit(train_x, train_y, **fit_params)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

In [ ]:
model = lgb.LGBMClassifier(colsample_bytree= 0.4, min_child_samples= 50, min_child_weight= 1, 
                           num_leaves= 50, reg_alpha= 50, reg_lambda= 0, subsample= 0.2)
model.fit(train_x, train_y)


In [ ]:
############## Model Save & Load ###########
filehandler = open("LGBoost.sav","wb")
pickle.dump(model,filehandler)
filehandler.close()

file = open("LGBoost.sav",'rb')
model= pickle.load(file)
file.close()